In [1]:
import os, sys
import string
import pandas as pd
try:
    import nltk
except:
    !conda install --yes --prefix {sys.prefix} nltk
    import nltk

In [2]:
# ver conteúdos da pasta
os.listdir()

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'BOA_project_description.pdf',
 'corpora',
 'corpora_preprocessing.ipynb',
 'LICENSE',
 'README.md']

In [3]:
authors = os.listdir("corpora/train")
authors

['AlmadaNegreiros',
 'CamiloCasteloBranco',
 'EcaDeQueiros',
 'JoseRodriguesSantos',
 'JoseSaramago',
 'LuisaMarquesSilva']

In [4]:
allfiles = []

for author in authors:
    for excerto in os.listdir(f"corpora/train/{author}"):
        allfiles.append(f"corpora/train/{author}/{excerto}")

allfiles

['corpora/train/AlmadaNegreiros/pg22615.txt',
 'corpora/train/AlmadaNegreiros/pg22730.txt',
 'corpora/train/AlmadaNegreiros/pg22801.txt',
 'corpora/train/AlmadaNegreiros/pg22802.txt',
 'corpora/train/AlmadaNegreiros/pg22969.txt',
 'corpora/train/AlmadaNegreiros/pg23133.txt',
 'corpora/train/AlmadaNegreiros/pg23620.txt',
 'corpora/train/AlmadaNegreiros/pg23879.txt',
 'corpora/train/AlmadaNegreiros/pg23961.txt',
 'corpora/train/CamiloCasteloBranco/24691-0.txt',
 'corpora/train/CamiloCasteloBranco/34756-0.txt',
 'corpora/train/CamiloCasteloBranco/pg16425.txt',
 'corpora/train/CamiloCasteloBranco/pg17927.txt',
 'corpora/train/CamiloCasteloBranco/pg19375.txt',
 'corpora/train/CamiloCasteloBranco/pg21406.txt',
 'corpora/train/CamiloCasteloBranco/pg23203.txt',
 'corpora/train/CamiloCasteloBranco/pg23345.txt',
 'corpora/train/CamiloCasteloBranco/pg23346.txt',
 'corpora/train/CamiloCasteloBranco/pg24339.txt',
 'corpora/train/CamiloCasteloBranco/pg25844.txt',
 'corpora/train/CamiloCasteloBranco/

In [5]:
# Pensar numa forma de compilar dataset de treino apenas num sítio (ex: pandas.DataFrame?)

preserve_blank_lines = True # mudar para False para excluir linhas em branco
list_of_texts = []
list_of_authors = []

for author in authors:
    for excerto in os.listdir(f"corpora/train/{author}"):
        text_list = []
        with open(f"corpora/train/{author}/{excerto}", encoding="utf-8") as f:
            for line in f:
                inner_list = [line.strip() for line in line.split("split character")]
                if preserve_blank_lines:
                    text_list.append(inner_list[0])
                else:
                    if len(inner_list[0]) > 0:
                        text_list.append(inner_list[0])
        list_of_texts.append(text_list)
        list_of_authors.append(author)
        

In [6]:
train = pd.DataFrame({"text":list_of_texts, "author":list_of_authors})

train

,text,author
0,"[Title: A Scena do Odio, , Author: José de Alm...",AlmadaNegreiros
1,"[Title: O Jardim da Pierrette, , Author: José ...",AlmadaNegreiros
2,"[, , Title: A Invenção do Dia Claro, , Author:...",AlmadaNegreiros
3,"[, Title: Litoral, A Amadeo de Souza Cardozo, ...",AlmadaNegreiros
4,"[, , , EXPOSIÇÃO, , +amadeo, de souza, cardoso...",AlmadaNegreiros
...,...,...
58,"[Título, e-medo, , Autora (inspiradíssima), Lu...",LuisaMarquesSilva
59,"[LISBOA 2050, Lisboa, ano de 2050. Um Agosto t...",LuisaMarquesSilva
60,"[Título, Um passeio pelo inferno, , Autora, Lu...",LuisaMarquesSilva
61,"[Título, Rapsódia sem dó (maior), , Autora, Lu...",LuisaMarquesSilva


In [7]:
train["author"].unique().tolist()

['AlmadaNegreiros',
 'CamiloCasteloBranco',
 'EcaDeQueiros',
 'JoseRodriguesSantos',
 'JoseSaramago',
 'LuisaMarquesSilva']

In [8]:
def remove_punctuation(text):
    """
        Greedy removal of all the punctuation from a list of text lines.
        However, it also removes the email and website punctuation, making
        them hard to recognise.
        Returns a list of text lines without punctuation.
    """
    no_punct_text = []
    for line in text:
        no_punct = "".join([char for char in line if char not in string.punctuation])
        no_punct_text.append(no_punct)
    return no_punct_text

In [9]:
# cada vez que uma nova função for criada, introduzi-la em "clean_text()"

def clean_text(df, text_col=str):
    """
        Compiles all the preprocessing functions inside a single function.
    """
    df[text_col] = df[text_col].apply(remove_punctuation)
    return df

In [10]:
# codificação da coluna y para fornecer ao futuro modelo

def target_encoding(df, target_col=str):
    """
        Encode the target column, i.e.:
            - AlmadaNegreiros     -> 0;
            - CamiloCasteloBranco -> 1;
            ...
    """
    pass

In [11]:
train = clean_text(train, "text")
train

,text,author
0,"[Title A Scena do Odio, , Author José de Almad...",AlmadaNegreiros
1,"[Title O Jardim da Pierrette, , Author José de...",AlmadaNegreiros
2,"[, , Title A Invenção do Dia Claro, , Author J...",AlmadaNegreiros
3,"[, Title Litoral, A Amadeo de Souza Cardozo, ,...",AlmadaNegreiros
4,"[, , , EXPOSIÇÃO, , amadeo, de souza, cardoso,...",AlmadaNegreiros
...,...,...
58,"[Título, emedo, , Autora inspiradíssima, Luísa...",LuisaMarquesSilva
59,"[LISBOA 2050, Lisboa ano de 2050 Um Agosto tão...",LuisaMarquesSilva
60,"[Título, Um passeio pelo inferno, , Autora, Lu...",LuisaMarquesSilva
61,"[Título, Rapsódia sem dó maior, , Autora, Luís...",LuisaMarquesSilva


[in progress] ***brainstorm* de features interessantes:**
- número de caracteres por linha
- número de caracteres especiais por linha
- número de vírgulas por frase
- número de reticências por linha
